## Readme
This notebook demonstrates our tool that can answer questions on WHO (World Health Organization) data. It takes in audio as input in the form of well structured questions and replies in accurate responses also in audio format. This is implemented using C# - and the Semantic Kernel SDK library.

We present 3 different ways in which the data can be fed to the LLM model.
1. In Tabular form
2. In Sentence form
3. Using the Data APIs directly

In [25]:
#r "nuget: Microsoft.SemanticKernel, 0.7.182.1-preview"
#r "nuget: Microsoft.CognitiveServices.Speech, 1.25.0"

using System.IO;
using Microsoft.SemanticKernel;
using Microsoft.SemanticKernel.CoreSkills;
using Microsoft.SemanticKernel.KernelExtensions;
using Microsoft.SemanticKernel.Orchestration;
using Microsoft.CognitiveServices.Speech;
using Microsoft.CognitiveServices.Speech.Audio;

var yourOpenAiKey = "<insert open-ai key>";
var yourSpeechKey = "<insert speech key>";
var yourSpeechRegion = "<insert speech region>";

Installed Packages Microsoft.CognitiveServices.Speech, 1.25.0 Microsoft.SemanticKernel, 0.7.182.1-preview

**LLM Model selection and Kernel building**

In [26]:
var kernel = Kernel.Builder.Build();
kernel.Config.AddOpenAICompletionBackend(
    label: "davinci",                     
    modelId: "text-davinci-003",                  
    apiKey: yourOpenAiKey       
);

**Skill-set selection**

In [28]:
var skillsDirectory = System.IO.Directory.GetCurrentDirectory();

var skill = kernel.ImportSemanticSkillFromDirectory(skillsDirectory, "skills");
var context = new ContextVariables();

### Use Case 1: Read data in tabular form

In this use case we input the tabular data in the raw format i.e. tab separated columns.

In [29]:
var dataset = @"Country / Region	Disease	2021	2020	2019	2018	2017	2016	2015	2014	2013	2012	2011	2010	2009	2008	2007	2006	2005	2004	2003	2002	2001	2000
Global	Mumps	""234,082""	""279,289""	""169,799""	""502,027""	""560,784""	""592,174""	""385,781""	""311,602""	""516,316""	""687,934""	""726,638""	""619,389""	""546,684""	""537,740""	""407,873""	""643,321""	""619,062""	""654,216""	""334,524""	""487,932""	""412,341""	""544,093""";

context.Set("results", dataset);

**Take speech input and convert to text**

In [33]:
Console.WriteLine("Speak into your microphone.");

string speechKey = yourSpeechKey;
string speechRegion = yourSpeechRegion;
var speechConfig = SpeechConfig.FromSubscription(speechKey, speechRegion);        
    speechConfig.SpeechRecognitionLanguage = "en-US";

var audioConfig = AudioConfig.FromDefaultMicrophoneInput();
var speechRecognizer = new SpeechRecognizer(speechConfig, audioConfig);
var speechRecognitionResult = await speechRecognizer.RecognizeOnceAsync();
OutputSpeechRecognitionResult(speechRecognitionResult);

static void OutputSpeechRecognitionResult(SpeechRecognitionResult speechRecognitionResult)
{
    switch (speechRecognitionResult.Reason)
    {
        case ResultReason.RecognizedSpeech:
            Console.WriteLine($"RECOGNIZED: Text={speechRecognitionResult.Text}");
            break;
        case ResultReason.NoMatch:
            Console.WriteLine($"NOMATCH: Speech could not be recognized.");
            break;
        case ResultReason.Canceled:
            var cancellation = CancellationDetails.FromResult(speechRecognitionResult);
            Console.WriteLine($"CANCELED: Reason={cancellation.Reason}");

            if (cancellation.Reason == CancellationReason.Error)
            {
                Console.WriteLine($"CANCELED: ErrorCode={cancellation.ErrorCode}");
                Console.WriteLine($"CANCELED: ErrorDetails={cancellation.ErrorDetails}");
                Console.WriteLine($"CANCELED: Did you set the speech resource key and region values?");
            }
            break;
    }
}

Speak into your microphone.
RECOGNIZED: Text=Which years are the maximum number of mumps diseases?


**Feed query to LLM and convert result to speech**

In [34]:
var input = speechRecognitionResult.Text;
context.Set("input", input);

var result = await kernel.RunAsync(context, skill["QA_WHO_Skill"]);
Console.WriteLine(result);


// Output: Text to Speech
// The language of the voice that speaks.
speechConfig.SpeechSynthesisVoiceName = "en-US-JennyNeural"; 

using (var speechSynthesizer = new SpeechSynthesizer(speechConfig))
{
    // Get text from the console and synthesize to the default speaker.
    string text = result.ToString();

    var speechSynthesisResult = await speechSynthesizer.SpeakTextAsync(text);
    OutputSpeechSynthesisResult(speechSynthesisResult, text);
}

static void OutputSpeechSynthesisResult(SpeechSynthesisResult speechSynthesisResult, string text)
    {
        switch (speechSynthesisResult.Reason)
        {
            case ResultReason.SynthesizingAudioCompleted:
                Console.WriteLine($"Speech synthesized for text: [{text}]");
                break;
            case ResultReason.Canceled:
                var cancellation = SpeechSynthesisCancellationDetails.FromResult(speechSynthesisResult);
                Console.WriteLine($"CANCELED: Reason={cancellation.Reason}");

                if (cancellation.Reason == CancellationReason.Error)
                {
                    Console.WriteLine($"CANCELED: ErrorCode={cancellation.ErrorCode}");
                    Console.WriteLine($"CANCELED: ErrorDetails=[{cancellation.ErrorDetails}]");
                    Console.WriteLine($"CANCELED: Did you set the speech resource key and region values?");
                }
                break;
            default:
                break;
        }
    }

 The maximum number of mumps diseases were reported in 2011 with 726,638 cases.
Speech synthesized for text: [ The maximum number of mumps diseases were reported in 2011 with 726,638 cases.]


### Use Case 2: Read data in text form
In this use case we pass the sentences to the model.  

In [35]:
var dataset = @"The Mortality rate for 5-14 year-olds (probability of dying per 1000 children aged 5-14 years) in Luxembourg for the Year 2019 is 0.38 with a lower bound of 0.25 and an upper bound of 0.57
The Mortality rate for 5-14 year-olds (probability of dying per 1000 children aged 5-14 years) in Iceland for the Year 2019 is 0.48 with a lower bound of 0.32 and an upper bound of 0.7
The Mortality rate for 5-14 year-olds (probability of dying per 1000 children aged 5-14 years) in San Marino for the Year 2019 is 0.51 with a lower bound of 0.4 and an upper bound of 0.65
The Mortality rate for 5-14 year-olds (probability of dying per 1000 children aged 5-14 years) in Denmark for the Year 2019 is 0.58 with a lower bound of 0.52 and an upper bound of 0.67
The Mortality rate for 5-14 year-olds (probability of dying per 1000 children aged 5-14 years) in Ireland for the Year 2019 is 0.61 with a lower bound of 0.5 and an upper bound of 0.71";
var context = new ContextVariables();
context.Set("results", dataset);


In [36]:
Console.WriteLine("Speak into your microphone.");
var speechRecognitionResult = await speechRecognizer.RecognizeOnceAsync();
OutputSpeechRecognitionResult(speechRecognitionResult);

context.Set("input", speechRecognitionResult.Text);

var result = await kernel.RunAsync(context, skill["QA_WHO_Skill"]);
Console.WriteLine(result);

// Text to Speech
using (var speechSynthesizer = new SpeechSynthesizer(speechConfig))
{
    // Get text from the console and synthesize to the default speaker.
    string text = result.ToString();

    var speechSynthesisResult = await speechSynthesizer.SpeakTextAsync(text);
    OutputSpeechSynthesisResult(speechSynthesisResult, text);
}

Speak into your microphone.
RECOGNIZED: Text=Which country has the highest mortality rate?
 Ireland has the highest mortality rate with a rate of 0.61 per 1000 children aged 5-14 years and a lower bound of 0.5 and an upper bound of 0.71.
Speech synthesized for text: [ Ireland has the highest mortality rate with a rate of 0.61 per 1000 children aged 5-14 years and a lower bound of 0.5 and an upper bound of 0.71.]


### Use Case 3: Read data from API
In this use case we directly pass the WHO provided Data API

In [39]:
var context = new ContextVariables();

string skPrompt = @"
These are the results from the API call {{$api}}
===RESULTS
{{$results}}
===END RESULTS

{{$resultsContext}}

Use the Results to answer the following query:

Query: {{$input}}
Answer:
";

var api = "https://ghoapi.azureedge.net/api/DIMENSION/COUNTRY/DimensionValues";
context.Set("api", api);
var results = "{\"Code\":\"ABW\",\"Title\":\"Aruba\",\"Dimension\":\"COUNTRY\",\"ParentDimension\":\"REGION\",\"ParentCode\":\"AMR\",\"ParentTitle\":\"Americas\"},{\"Code\":\"AFG\",\"Title\":\"Afghanistan\",\"Dimension\":\"COUNTRY\",\"ParentDimension\":\"REGION\",\"ParentCode\":\"EMR\",\"ParentTitle\":\"Eastern Mediterranean\"},{\"Code\":\"AGO\",\"Title\":\"Angola\",\"Dimension\":\"COUNTRY\",\"ParentDimension\":\"REGION\",\"ParentCode\":\"AFR\",\"ParentTitle\":\"Africa\"}";

Console.WriteLine("Speak into your microphone.");
var speechRecognitionResult = await speechRecognizer.RecognizeOnceAsync();
OutputSpeechRecognitionResult(speechRecognitionResult);

context.Set("input", speechRecognitionResult.Text);
context.Set("results", results);

var qaFunction = kernel.CreateSemanticFunction(skPrompt, maxTokens: 200, temperature: 0, topP: 0.5);

var answer = await kernel.RunAsync(
    context,
    qaFunction);
    
    Console.WriteLine(answer);

    using (var speechSynthesizer = new SpeechSynthesizer(speechConfig))
{
    string text = answer.ToString();

    var speechSynthesisResult = await speechSynthesizer.SpeakTextAsync(text);
    OutputSpeechSynthesisResult(speechSynthesisResult, text);
}

Speak into your microphone.
RECOGNIZED: Text=Give me some key insights from the API response.
The API response provides information about three countries: Aruba, Afghanistan, and Angola. Each country is associated with a region, with Aruba being in the Americas, Afghanistan in the Eastern Mediterranean, and Angola in Africa.
Speech synthesized for text: [The API response provides information about three countries: Aruba, Afghanistan, and Angola. Each country is associated with a region, with Aruba being in the Americas, Afghanistan in the Eastern Mediterranean, and Angola in Africa.]
